# Hf_Emptions Predictor

In [6]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

  Using cached sagemaker-2.77.1.tar.gz (513 kB)
  Preparing metadata (setup.py) ... done
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached sagemaker-2.77.0.tar.gz (513 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.76.0.tar.gz (512 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.75.1.tar.gz (511 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.75.0.tar.gz (511 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.74.0.tar.gz (481 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.73.0.tar.gz (481 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.3.tar.gz (475 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.2.tar.gz (473 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.1.tar.gz (473 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.0.tar.gz (477 kB)
  Pre

In [7]:
import sagemaker.huggingface

In [8]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::647333706880:role/service-role/AmazonSageMaker-ExecutionRole-20210125T093214
sagemaker bucket: sagemaker-us-east-1-647333706880
sagemaker session region: us-east-1


## get and deploy model

In [12]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 


In [15]:
# model location comes from end of `Hf_EmotionsEstimator` notebook
# but we must add a second '/' to 's3:/''
model_loc = 's3://sagemaker-us-east-1-647333706880/huggingface-pytorch-training-2022-02-28-18-47-41-609/output/model.tar.gz'

In [16]:
# create Hugging Face Model Class

huggingface_model = HuggingFaceModel(
   model_data=model_loc,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [64]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [18]:
# quick predictor test
sentiment_input= {"inputs":"I love using the new Inference DLC."}

predictor.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9884716272354126}]

### delete endpoint

In [93]:
# delete when finished
predictor.delete_endpoint()

## get test,train, validation data
Paths created and data stored in `Hf_EmotionEstimator`

In [27]:
from datasets import load_from_disk
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

In [31]:
[training_input_path, test_input_path,validation_input_path] = ['s3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/train',
 's3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/test',
 's3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/validation']

In [35]:
train_dataset = load_from_disk(training_input_path,fs=s3)
test_dataset = load_from_disk(test_input_path,fs=s3)
validation_dataset = load_from_disk(validation_input_path,fs=s3)


## start aws: create an awstransformer to run batches of input throught the model
### use a dataset (test or validation) depending on fit to test the model

see 
* [here](https://go.aws/3tieUp7)
* [Increasing the timeout for InvokeEndpoint · Issue #1119 · aws/sagemaker-python-sdk](https://bit.ly/3vA6jAK)

Status:
csv file doesnt seem formatted correctly. 
error is: 
* `2022-03-01 23:15:39,271 [INFO ] W-model-2-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -     inputs = data.pop("inputs", data)`
* `2022-03-01T23:15:39.284:[sagemaker logs]: sagemaker-us-east-1-647333706880/samples/datasets/emotion/transforms/validation.csv:   "message": "\u0027numpy.ndarray\u0027 object has no attribute \u0027pop\u0027"`

In [83]:
x = validation_dataset["text"].to_csv('validation.csv',index=False,header=False)

In [86]:
import boto3,os
s3_prefix = 'samples/datasets/emotion'
s3_transform_prefix = s3_prefix + "/transforms"
[sess.default_bucket(),s3_prefix,s3_transform_prefix]

['sagemaker-us-east-1-647333706880',
 'samples/datasets/emotion',
 'samples/datasets/emotion/transforms']

In [87]:
# Load file to s3 for transformer
boto3.Session().resource('s3').Bucket(sess.default_bucket()).Object(
    os.path.join(s3_transform_prefix, 'validation.csv')).upload_file('validation.csv')

In [92]:
# The location of the test dataset
batch_input = 's3://{}/{}'.format(sess.default_bucket(), s3_transform_prefix)

# The location to store the results of the batch transform job
batch_output = 's3://{}/{}/batch-prediction'.format(sess.default_bucket(), s3_transform_prefix)
[batch_input,batch_output]

['s3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/transforms',
 's3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/transforms/batch-prediction']

In [94]:
transformer = huggingface_model.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge', 
    output_path=batch_output
)

In [95]:
transformer.transform(
    data=batch_input, 
    data_type='S3Prefix',
    content_type='text/csv', 
    split_type='Line'
)
transformer.wait()

..........................................
.2022-03-01T23:15:28.810:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
2022-03-01 23:15:27,709 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
2022-03-01 23:15:27,709 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 2748 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
MMS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 2748 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sag

UnexpectedStatusException: Error for Transform job huggingface-pytorch-inference-2022-03-01-23-08-39-791: Failed. Reason: ClientError: See job logs for more information

## end aws: create an awstransformer to run batches of input throught the model

## Hugging face batch transformer
* [youtube](https://bit.ly/3K2YmIi)
* [notebook](https://bit.ly/3pvF4n1)

In [98]:
import csv
import json
import sagemaker
from sagemaker.s3 import S3Uploader,s3_path_join


In [109]:
import pandas as pd
import io

In [86]:
import boto3,os
s3_prefix = 'samples/datasets/emotion'
s3_transform_prefix = s3_prefix + "/transforms"
[sess.default_bucket(),s3_prefix,s3_transform_prefix]

['sagemaker-us-east-1-647333706880',
 'samples/datasets/emotion',
 'samples/datasets/emotion/transforms']

In [101]:
sagemaker_session_bucket = sess.default_bucket()

In [122]:
# datset files
dataset_csv_file="validation.csv"
dataset_jsonl_file="validation.jsonl"


In [159]:
x = validation_dataset["text"].to_csv(index=False,header=False)
x="inputs\n" + x
buffer = io.StringIO(x)

xdf = pd.read_csv(filepath_or_buffer = buffer, header = 0)
xdf.to_csv(dataset_csv_file,index=False,header=1)

In [147]:
with open(dataset_csv_file, "r+") as infile, open(dataset_jsonl_file, "w+") as outfile:
    reader = csv.DictReader(infile)
    for row in reader:
        # remove @
        #row["inputs"] = row["inputs"].replace("@","")
        json.dump(row, outfile)
        outfile.write('\n')


In [149]:
# uploads a given file to S3.
input_s3_path = s3_path_join("s3://",sess.default_bucket(), s3_transform_prefix)
output_s3_path = s3_path_join("s3://",sess.default_bucket(), s3_transform_prefix)
s3_file_uri = S3Uploader.upload(dataset_jsonl_file,input_s3_path)


In [127]:
print(f"{dataset_jsonl_file} uploaded to {s3_file_uri}")

validation.jsonl uploaded to s3://sagemaker-us-east-1-647333706880/samples/datasets/emotion/transforms/validation.jsonl


In [150]:
# create Transformer to run our batch job
# ss this run
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    output_path=output_s3_path, # we are using the same s3 path to save the output with the input
    strategy='SingleRecord')

In [152]:
# create Transformer to run our batch job
#ss try multirecord --not yet
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    output_path=output_s3_path, # we are using the same s3 path to save the output with the input
    strategy='MultiRecord')

In [153]:
# starts batch transform job and uses s3 data as input
batch_job.transform(
    data=s3_file_uri,
    content_type='application/json',    
    split_type='Line')

.........................................2022-03-02 05:16:50,768 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 1
Number of CPUs: 8
Max heap size: 12948 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 1
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Preload model: false
Prefer direct buffer: false
2022-03-02 05:16:50,862 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-9000-model
2022-03-02 05:16:50,995 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - model_service_worker started with ar

UnexpectedStatusException: Error for Transform job huggingface-pytorch-inference-2022-03-02-05-10-05-104: Failed. Reason: ClientError: See job logs for more information

## End Hugging face batch transformer


## testing

In [162]:
x = validation_dataset["text"].to_csv(index=False,header=False)
#x="inputs\n" + x
buffer = io.StringIO(x)

xdf = pd.read_csv(filepath_or_buffer = buffer, header = 0)
xdf.to_csv("test.csv",index=False,header=1)

In [166]:
with open("test.csv", "r+") as infile, open("test.jsonl", "w+") as outfile:
    reader = csv.reader(infile)
    ro = '{"inputs": ['
    for row in reader:
        # remove @
        #row["inputs"] = row["inputs"].replace("@","")
        ro =ro +'"'+row[0]+'"'+"\n"
    
    ro = ro + "]}"
with  open("test.jsonl", "w+") as outfile:
    json.dump(ro, outfile)
    outfile.write('\n')


In [37]:
# we used train and test to fit so use validation
def dataset_label_int2str(row):
    return validation_dataset.features["labels"].int2str(row)


In [40]:
validation_dataset.set_format(type="pandas")
df = validation_dataset[:]


df["label_name"] = df["labels"].apply(dataset_label_int2str)
df.head()

,attention_mask,input_ids,labels,text,label_name
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2123, 2102, 2113, 2021, 1045, 2514...",1,i dont know but i feel virtuous so i accept th...,joy
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 5791, 2514, 2045, 1055, 2070, 6179...",1,i definitely feel there s some useful informat...,joy
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2514, 2004, 2295, 1045, 2031, 6414...",1,i feel as though i have merely accepted what h...,joy
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2514, 2008, 2002, 2003, 16448, 203...",2,i feel that he is gazing me and giving a naugh...,love
4,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 1045, 2074, 2514, 8616, 102, 0, 0, 0, 0,...",2,i just feel tender,love


In [56]:
input_list = validation_dataset["text"].to_list()

In [67]:
# too much input results in error: https://bit.ly/3hrdWBu
input_list = validation_dataset["text"].to_list()
#inputs= {"inputs":"this is bullshit"}
inputs= {"inputs":input_list[:1500]}
preds =predictor.predict(inputs)
preds

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2022-03-01-20-49-25-193 in account 647333706880 for more information.